---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [46]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [47]:
pd.set_option('display.max_rows', 500)

def date_sorter():
    global df 
 
    pattern1 = '(?P<OriginalMatch>(?P<Month>\d?\d)[/|-](?P<Day>\d?\d)[/|-](?P<Year>\d{2,4}))'
    
    pattern2 =  '(?P<OriginalMatch>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[\S]*[^a-zA-Z\d:](?P<Day>\d{1,2})[\S]*[^a-zA-Z\d:](?P<Year>\d{4}))'  

    pattern3 = '(?P<OriginalMatch>(?P<Day>\d{1,2})[+\s|-](?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[\S]*[\s|-](?P<Year>\d{4}))'

    pattern4 = '(?P<OriginalMatch>(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[\S]*[\s](?P<Day>\d{1,2})[\S]*[,][\s](?P<Year>\d{4}))'


    pattern5 = '(?P<OriginalMatch>[^\d{1,2}]?\S?(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[\S]*[\s|-](?P<Year>\d{4}))'
   

    pattern6 = '(?P<OriginalMatch>[\s|^/|\~|\.|\(|\|)|\d/]?(?P<Month>\d{1,2})/(?P<Year>\d{4}))'

    pattern7 = '(?P<OriginalMatch>(?<!/)[^/|\s|\~|\.|\(|\|)]?(?P<Year>[1|2]\d{3})(?!\d{1,2}/))'


#     df = pd.DataFrame(doc, columns=['text'])
    
    month_dictionary = {
        'Jan': 1,
        'Feb': 2,
        'Mar': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12
    }

    
    df_pattern1 = df.str.extract(pattern1)
    
    df_pattern1 = df_pattern1.dropna()
    
    pattern1_indexes = df_pattern1.index.values.tolist()

    df_pattern1["Year"] = df_pattern1['Year'].apply(lambda x: '19'+str(x) if len(x) == 2 else x)
    
    df_pattern1['Year'] = df_pattern1['Year'].apply(lambda x: int(x))

    df = df.drop(pattern1_indexes) 
    
    df_pattern1['Pattern'] = 'Pattern 1'
    
    
    df_pattern2 = df.str.extract(pattern2)
    
    df_pattern2 = df_pattern2.dropna()
    
    df_pattern2_indexes = df_pattern2.index.values.tolist()
    
    df_pattern2["Month"] = df_pattern2['Month'].apply(lambda x: month_dictionary[x])
    
    df_pattern2['Pattern'] = 'Pattern 2'
    
    df = df.drop(df_pattern2_indexes) 
    
    df_pattern3 = df.str.extract(pattern3)
    
    df_pattern3 = df_pattern3.dropna()
    
    df_pattern3["Month"] = df_pattern3['Month'].apply(lambda x: month_dictionary[x])
    
    df_pattern3_indexes = df_pattern3.index.values.tolist()
    
    df_pattern3['Pattern'] = 'Pattern 3'
    
    df = df.drop(df_pattern3_indexes) 
    
    df_pattern4 = df.str.extract(pattern4)
    
    df_pattern4 = df_pattern4.dropna()
    
    df_pattern4_indexes = df_pattern4.index.values.tolist()
    
    df_pattern4['Pattern'] = 'Pattern 4'
    
    df = df.drop(df_pattern4_indexes) 
    df_pattern5 = df.str.extract(pattern5)
    
    df_pattern5 = df_pattern5.dropna()
    
    df_pattern5["Month"] = df_pattern5['Month'].apply(lambda x: month_dictionary[x])
    
   
    df_pattern5['Day']  = 1
    
    df_pattern5_indexes = df_pattern5.index.values.tolist()
    
    df_pattern5['Pattern'] = 'Pattern 5'
    
    df = df.drop(df_pattern5_indexes) 
    df_pattern6 = df.str.extract(pattern6)
    
    df_pattern6["Month"] = df_pattern6['Month'].apply(lambda x: 10 if str(x) == '0' else x)
    
    
    df_pattern6 = df_pattern6.dropna()
    
    df_pattern6['Day']  = 1
    
    df_pattern6_indexes = df_pattern6.index.values.tolist()
    
    df_pattern6['Pattern'] = 'Pattern 6'
    
    df = df.drop(df_pattern6_indexes) 
    df_pattern7 = df.str.extract(pattern7)
    
    df_pattern7 = df_pattern7.dropna()
    
    df_pattern7['Day']  = 1
    df_pattern7['Month']  = 1
    df_pattern7_indexes = df_pattern7.index.values.tolist()
    df_pattern7['Pattern'] = 'Pattern 7'
    df = df.drop(df_pattern7_indexes) 
    
    
    
    combined_df = pd.concat([df_pattern1, df_pattern2, df_pattern3, df_pattern4,
                               df_pattern5, df_pattern6, df_pattern7], ignore_index=False, sort=True)
    
    combined_df['Month'] = combined_df['Month'].apply(lambda x: str(x))
    
    combined_df['Year'] = combined_df['Year'].apply(lambda x: str(x))
    
    combined_df['Day'] = combined_df['Day'].apply(lambda x: str(x))
    
    combined_df['FormatedDate'] = combined_df['Month'] + '/' + combined_df['Day'] + '/' + combined_df['Year']
    
    
    combined_df['Date'] = pd.to_datetime(combined_df['FormatedDate'])
    
#     combined_df['datetime'] = (pd.to_datetime(str(combined_df['Year']) + str(combined_df['Month']) + str(combined_df['Day']), format='%Y%B%d'))

    combined_df.sort_values(by='Date', inplace=True)
    
#     sorted_df = combined_df.sort_values(by=['Year', 'Month', 'Day'])
    combined_df = combined_df.reset_index()
    s = pd.to_numeric(combined_df.iloc[:,0])
    s = pd.Series(s)
 
    return s
date_sorter()


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
30     162
31     154
32     402
33      95
34      73
35     108
36     156
37     332
38     182
39      82
40     351
41     278
42     214
43     155
44     223
45     473
46     363
47      49
48     317
49      11
50     319
51      40
52     418
53     165
54     370
55     382
56       3
57      50
58     219
59     465
60     237
61      23
62     342
63     204
64     258
65     315
66      27
67      93
68      17
69     303
70     488
71     395
72     283
73     309
74     419
75     123
76      19
77     117
78     232
79      72
80     189
81     318
82     493
83     369
84     239
85     148
86     105
87     336
88       6
89     200
90      81

In [29]:
date_sorter()

TypeError: unsupported operand type(s) for +: 'int' and 'str'